<a href="https://www.kaggle.com/code/ibhong/card-fraud-detection-ilbunghong?scriptVersionId=239468853" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 1. 초기 환경설정

In [ ]:
!pip install -U scikit-learn==1.4.2 imbalanced-learn==0.12.0

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# 2. Data Loading

In [ ]:
train = pd.read_csv("/kaggle/input/modu-ds-4-credit-card-fraud-detection/train.csv")
test = pd.read_csv("/kaggle/input/modu-ds-4-credit-card-fraud-detection/test.csv")

# 3. EDA

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
train.head()

In [ ]:
test.head()

## Check the columns

In [ ]:
train.columns

In [ ]:
test.columns

In [ ]:
for column in train.columns:
    if column not in test.columns:
        print(column)

* Target column = test['Class'] = prediction

In [ ]:
train["Class"].value_counts(normalize=True) * 100

In [ ]:
train["Class"].value_counts() 

In [ ]:
Train_count = train["Class"].value_counts().iloc[1]
Total_count = round((len(train) + len(test)) * 0.00172)
Prediction_count = Total_count - Train_count
print(Train_count,Prediction_count,Total_count)

In [ ]:
train_fraud_ratio = Train_count / len(train)
test_fraud_ratio = Prediction_count / len(test)
total_fraud_ratio = Total_count / (len(train) + len(test))
print(train_fraud_ratio,test_fraud_ratio,total_fraud_ratio)

## Target Analysis

* Train Fraud Count = 360 (관측치)
* Total Fraud Count = 130 (예상치)
* Total Fraud Count = 490 (예상치)
* Train Fraud Ratio = 0.211% (관측치)
* Test  Fraud Ratio = 0.114% (예상치)
* Total Fraud Ratio = 0.172% (Meta-data)
> Metadata: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud

### 분석 결과
Train Data Set에서 0.21% 정도의 사기 데이터가 포함된 반면, 메타데이터가 정확하다면 Test Data Set에서는 해당 수치의 약 절반 수준인 0.11%의 Fraud 데이터만을 포함하고 있을 것으로 예상된다. 이는 데이터 셋이 'Class'칼럼을 기준으로 매우 train data와 test data에 불균형적으로 분포하고 있음을 시사한다. 

## Check the slicing Ratio

In [ ]:
print (f"Train data size:\t{train.shape[0]}")
print (f"Test data size: \t{test.shape[0]}")
print (f"Total data size:\t{train.shape[0] + test.shape[0]}")
print (f"Test-Train Ratio:\t{test.shape[0] / train.shape[0] * 100:.4}%")


*  Test : Train = 2 : 3

In [ ]:
# 170882 를 기점으로 그 뒤 시점으로 자름

train["id"].max(), test["id"].min()

* Finding: Sorted data set by column 'id'

## Define the total dataset of X

In [ ]:
total_X = pd.concat([train.drop('Class', axis = 1), test])

In [ ]:
total_X.head()

In [ ]:
total_X.tail(5)

In [ ]:
total_X.info()

In [ ]:
total_X.describe()

In [ ]:
import missingno as msno
msno.bar(df = total_X)

## 전체 데이터 특징 요약
* 데이터 건수: 284,806건
* 결측치 비중: 0%
* 독립변수(X): 연속형 데이터
* 종속변수(y): 범주형 데이터

## 독립변수 상관관계 분석

In [ ]:
corr = total_X.drop(['id','Time'], axis = 1).corr()
corr

In [ ]:
sns.heatmap(corr,
            cbar = True,
            cmap = 'RdBu'
           )
plt.show()

In [ ]:
corr['Amount'].sort_values(ascending = False).head(5)

## 상관관계 분석 결과
* 시각적으로 확인했을 때 Amount칼럼은 각 V1~V27변수들과 모종의 상관관계가 있다.
* Amount칼럼과 상대적으로 높은 상관관계를 보이는 칼럼은['V7','V20','V21']이다.
* 독립변수 V1~V27 간에는 특별히 다중공선성의 문제는 발견되지 않는다. 

## V1~V28 분산 시각화

In [ ]:
import math

# 숫자형 칼럼만 선택
df = total_X.drop(['id','Time','Amount'], axis = 1)
numeric_cols = df.columns
n = len(numeric_cols)

cols = 4  # 한 줄에 4개
rows = math.ceil(n / cols)

plt.figure(figsize=(cols * 4, rows * 3))

for i, col in enumerate(numeric_cols):
    plt.subplot(rows, cols, i + 1)
    plt.hist(df[col], bins=50, color='skyblue', edgecolor='black')
    plt.title(col)
    plt.tight_layout()

plt.suptitle("Distribution of Numeric Columns", fontsize=16, y=1.02)
plt.show()

## 칼럼 Amount 분석

In [ ]:
sns.histplot(x='Amount', data=train, bins=100, kde=True, hue = 'Class')
plt.show()

In [ ]:
df_sorted = train.sort_values(by = 'Class')
sns.scatterplot(
                x = 'id',
                y = 'Amount',
                hue = 'Class', 
                data = df_sorted,
                alpha = 0.3,                  
                palette = {0: 'gray', 1: 'red'}, 
                s = 20                     
                )
plt.show()

In [ ]:
sns.boxplot(
            x = 'Class',
            y = 'Amount',
            data = train
            )
plt.show()

In [ ]:
train['Amount'].describe()

In [ ]:
train['Amount'].quantile([0.2, 0.4, 0.6, 0.8])

### Amount 칼럼 분석

* 다른 연속형 칼럼과는 다르게 비정규화된 형식을 띄고 있음.
* 다른 칼럼과 유사하게 정규화 처리를 거칠 필요가 있음.
* 카드 사용금액이 100불 이하인 데이터가 주를 이루고 있음.
* 일부 고객은 카드 사용금액이 주류 고객보다 월등하게 많음.
* 선형 알고리즘 적용 시엔 해당 칼럼에서 이상치를 제거할 필요가 있음. 

 # 4. Function Definition

In [ ]:
# 평가 함수

def get_clf_eval(y_test, pred, pred_proba=None):
    
    # input: test data, model prediction
    # output: confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
    
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_auc = roc_auc_score(y_test, pred_proba) # 최종평가지표
    
    print("오차 행렬")
    print(confusion)
    print(f"정확도: {accuracy:.4f}, 정밀도: {precision: .4f}, 재현율: {recall: .4f}, f1스코어: {f1:.4f}, roc-auc: {roc_auc:.4f}")

In [ ]:
def get_model_train_eval(model, ftr_train=None, ftr_test=None, tgt_train=None, tgt_test=None):
    
    # input: fractional train data set
    # output: evaluation index prints
    
    model.fit(ftr_train, tgt_train)
    pred = model.predict(ftr_test)
    pred_proba = model.predict_proba(ftr_test)[:, 1] #id 제외한 나머지 X
    get_clf_eval(tgt_test, pred, pred_proba)

# 5. Preprocess

* Amount: 스케일링 (Scaling), 이상치 처리
* Time: 제거
* id: 제거


## Outliar 처리함수 정의

In [ ]:
def get_outlier(df=None, column=None, weight=1.5, option='index'):

    # input: train data, column name, weight(optional)
    # output: index of Outliars of Fraud records

    fraud = df[df["Class"] == 1][column]
    quantile_25 = np.percentile(fraud.values, 25) # 1분위값
    quantile_75 = np.percentile(fraud.values, 75) # 3분위값
    
    iqr = quantile_75 - quantile_25 # Interquartile Range
    iqr_weight = iqr * weight
    lowest_val = quantile_25 - iqr_weight
    highest_val = quantile_75 + iqr_weight

    normal_range = ~(fraud < lowest_val) | (fraud > highest_val) # About 99.3% in Normal Distribution
    
    outlier_index = fraud[~normal_range].index
    if(option == 'index'):
        return outlier_index
    else:
        return lowest_val,highest_val

In [ ]:
outlier_index = get_outlier(df = train, column = 'Amount', weight = 1.5) # Not Normalized
print('이상치 데이터 인덱스:', outlier_index)

* Fraud 데이터 중에는 Amount 기준으로 Outliar에 해당되는 레코드가 없음.

In [ ]:
sns.boxplot(
            x = 'Class',
            y = 'V14',
            data = train
            )
plt.show()

### V14칼럼 4분위값 분석

* Fraud 데이터와 일반 고객 데이터의 V14 분포범위가 다르게 나타남.
* Fraud 데이터 중에서 V14값이 비교적으로 낮은 Outliar가 관측됨.

In [ ]:
outlier_index = get_outlier(df = train, column = 'V14', weight = 1.5)
print('이상치 데이터 인덱스:', outlier_index)

* Fraud 데이터 중에는 V14 기준으로 Outliar에 해당되는 레코드가 3건 존재.

## 전처리 이전 모델 성능 측정

### HyperParameter Default Setting

In [ ]:
lr_clf = LogisticRegression(max_iter = 1000)
lgbm_clf = LGBMClassifier(n_estimators = 1000, 
                          num_leaves = 64, 
                          n_jobs = -1, 
                          boost_from_average = False,
                          verbose = -1,
                          random_state = 123
                         )

### Train Data Slicing

In [ ]:
# 사전 데이터 가공 후 학습과 테스트 데이터 세트를 반환하는 함수

def get_train_test_dataset(df = train):
    
    # input: Train Data Set
    # output: X_train, X_test, y_train, y_test

    # df_copy = get_preprocessed_df(df) # 필요할 때 활성화
    df_copy = df
    X_features = df_copy.iloc[:, :-1] # X : Columns except target column
    y_target = df_copy.iloc[:, -1] # Target : Class
    
    X_train, X_test, y_train, y_test = train_test_split(X_features, y_target,
                                                        test_size = 0.3, # train:test = 7:3
                                                        random_state = 0, 
                                                        stratify = y_target
                                                       )
    
    return X_train, X_test, y_train, y_test

In [ ]:
# Train Data set 전처리 및 Slicing
X_train, X_test, y_train, y_test = get_train_test_dataset()
print('### 로지스틱 회귀 예측 성능 ###')
get_model_train_eval(lr_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)
print('### LightGBM 예측 성능 ###')
get_model_train_eval(lgbm_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

### 전처리 이전 모델링 결과 분석

* 정확도 기준으로 두 모델 모두 99.9% 이상의 예측을 정확히 수행해냄.
* roc-auc 점수 기준으로 LightGBM 모델의 성능이 비교적 우수함.

## 전처리 이후 모델링 결과 분석

## 전처리 함수 정의

* 불필요 칼럼 제거: Amount, Time
* 스케일링: Amount 칼럼 로그 스케일링 > V29
* 이상치 제거: V14 칼럼 기준


> np.log1p(x): log1p(x)=log e• (1+x)


In [ ]:
def get_preprocessed_df(df = None, train = True):
    
    # input: Datafram with cloumn ["Amount","Time"]
    # output: Datafram with log_scaled column "Amount", without "Time"

    df_copy = df.copy()
    amount_n = np.log1p(df_copy["Amount"])
    df_copy.insert(0, "V29", amount_n) # V29 : Amount_Scaled
    df_copy.drop(["Time", "Amount"], axis = 1, inplace = True)
    
    outlier_index = get_outlier(df = df_copy, column = "V14", weight = 1.5)
    df_copy.drop(outlier_index, axis = 0, inplace = True) #V14 Outliar 제거
    
    return df_copy

### Prepross and slice again

In [ ]:
train_preprocessed = get_preprocessed_df(train)

In [ ]:
train_preprocessed.columns

In [ ]:
sns.histplot(x='V29', data=train_preprocessed, bins=100, kde=True, hue = 'Class')
plt.show()

Amount 칼럼 스케일링 결과: 정규분포에 가까운 분포형태를 띄게 됨.

In [ ]:
sns.boxplot(
            x = 'Class',
            y = 'V29',
            data = train_preprocessed
            )
plt.show()

In [ ]:
corr = train_preprocessed.drop('id', axis = 1).corr()
sns.heatmap(corr,cbar = True,cmap='RdBu')
plt.show()

## Medel Evaluation after preprossing

In [ ]:
X_train, X_test, y_train, y_test = get_train_test_dataset(df = train_preprocessed)

In [ ]:

print('### 로지스틱 회귀 예측 성능 ###')
get_model_train_eval(lr_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)
print('### LightGBM 예측 성능 ###')
get_model_train_eval(lgbm_clf, ftr_train=X_train, ftr_test=X_test, tgt_train=y_train, tgt_test=y_test)

### 전처리 이후 모델링 결과 분석

* 로지스틱 회귀 성능: 정확도 -0.0002%p, roc-auc: -0072%p
* LightGBM 예측 성능: 정확도 0, roc-auc: 0

# 6. SMOTE Over Sampling

* 지도학습 알고리즘 극도로 불균형한 레이블 값 분포의 문제 해결을 위한 학습 데이터셋 확보 방안
* Over Sampling: 적은 데이트 세트를 증식하여 학습을 위한 충분한 데이터를 확보하는 방법.
    * SMOTE: K 최근접 이웃(KNN)을 찾아서 기존 데이터와 약간 차이가 나는 새로운 데이터를 생성하는 기법

In [ ]:
print("학습 데이터 레이블 값 비율")
print(y_train.value_counts()/y_train.shape[0] * 100)
print()
print("테스트 데이터 레이블 값 비율")
print(y_test.value_counts()/y_test.shape[0] * 100)

* 기존의 Fraud 데이터 비중은 0.21% 수준으로 학습할 수 있는 데이터의 양이 매우 적은 수준

In [ ]:
smote = SMOTE(random_state = 0)

X_train_over, y_train_over = smote.fit_resample(X_train, y_train)

In [ ]:
print("SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ", X_train.shape, y_train.shape)
print("\nSMOTE 적용 후 학습용 피처/레이블 데이터 세트: ", X_train_over.shape, y_train_over.shape)
print("\nSMOTE 적용 후 레이블 값 분포: \n", pd.Series(y_train_over).value_counts(normalize=True))

In [ ]:
# ftr_train과 tgt_train 인자값이 SMOTE 증식된 X_train_over와 y_train_over로 변경됨에 유의
get_model_train_eval(lr_clf, 
                     ftr_train = X_train_over, 
                     ftr_test = X_test, 
                     tgt_train = y_train_over, 
                     tgt_test = y_test
                    )

## SMOTE Over-sampling 결과 분석

* Over-sampling 적용 결과 로지스틱 회귀 알고리즘의 roc-auc 스코어 4%p 증가
* 반면, 정확도는 Over-sampling 적용 이전에 비해 1%p 정도 감소.
* 신규 증식한 데이터 셋은 Train data set의 특징을 바탕으로 한 것이다.
* 결론적으로 오버샘플링은 Train data에 적합하게 학습을 진행한다. 

# 7. Modeling

## Model Selection

* 선택기준: 다중 연속형 독립변수를 통해 범주형 종속변수를 효율적으로 분류할 수 있는가
* 모델후보: LogisticRegression, LGBMClassifier
* model 변수에 실험 대상 모델을 적용하여 실험 진행.

## LogisticRegression

* Learning train data set and evaluate the model
* Learning over-sampled data set and evaluate the model

In [ ]:
get_model_train_eval(model = lr_clf, 
                     ftr_train = X_train, 
                     ftr_test = X_test, 
                     tgt_train = y_train, 
                     tgt_test = y_test
                    )

In [ ]:
get_model_train_eval(model = lr_clf, 
                     ftr_train = X_train_over, 
                     ftr_test = X_test, 
                     tgt_train = y_train_over, 
                     tgt_test = y_test
                    )

### 로지스틱 회귀분석 성능분석

* 정확도 측면에서 train 데이터 셋에 적용한 경우 성능이 보다 우수
* ROC_AUC 측면에서 over_sampled 데이터 셋에 적용한 경우 성능이 보다 4%p 우수
* 그러나 여전히 Train Data에 적합한 모델링 기법이라는 한계를 지님.
* Train data set과 Test 데이터 셋의 Class 분균형 해소여부 의문.

## LGBMClassifier

* Learning train data set and evaluate the model
* Learning over-sampled data set and evaluate the model

In [ ]:
get_model_train_eval(model = lgbm_clf, 
                     ftr_train = X_train, 
                     ftr_test = X_test, 
                     tgt_train = y_train, 
                     tgt_test = y_test)

In [ ]:
get_model_train_eval(model = lgbm_clf, 
                     ftr_train=X_train_over, 
                     ftr_test=X_test, 
                     tgt_train=y_train_over, 
                     tgt_test=y_test
                    )

### LGBMClassifier 성능분석

* 정확도 측면에서 train 데이터 셋에 적용한 경우 성능이 거의 유사.
* ROC_AUC 측면에서 over_sampled 데이터 셋에 적용한 경우 성능이 소폭 증가.
* 그러나 여전히 Train Data에 적합한 모델링 기법이라는 한계를 지님.
* Train data set과 Test 데이터 셋의 Class 분균형 해소여부 의문.

# 8.Prediction Submission

## Test data prepross

In [ ]:
test.head()

In [ ]:
test.V14.describe()

In [ ]:
normal_Amount = get_outlier(df = train, column = 'Amount', weight = 1.5, option = 'range')
normal_Amount
# test[(test['V14'] < normal_Amount[0]) | (test['V14'] > normal_Amount[1])]

## 결측치 분석
* test 데이터는 train 데이터에서 설정한 범주 밖의 레코드가 존재하지 않는다.

In [ ]:
test["V29"] = np.log1p(test["Amount"])

In [ ]:
test.drop(["Time", "Amount"], axis=1, inplace=True)

In [ ]:
test.columns

In [ ]:
test = test[[test.columns[-1]] + list(test.columns[:-1])]
test.columns

## Final Prediction by Logistic Regression Model

In [ ]:
lr_pred = lr_clf.predict

In [ ]:
test["Class"] = lr_pred

In [ ]:
test.Class.value_counts() / len(test)

In [ ]:
for column in train_preprocessed.columns:
    if column not in test.columns:
        print(column)

In [ ]:
total = pd.concat([test,train_preprocessed], axis = 0)

## Check distribution of V14 column

In [ ]:
# source 이름 부여
train['source'] = 'train'
test['source'] = 'test'
total['source'] = 'total'

# 필요한 칼럼만 남기고 병합
df_all = pd.concat([
    train[['V14', 'Class', 'source']],
    test[['V14', 'Class', 'source']],
    total[['V14', 'Class', 'source']]
], ignore_index=True)

# 시각화
plt.figure(figsize=(10, 6))
sns.boxplot(x='source', y='V14', hue='Class', data=df_all, whis=1.5) #weight = 1.5 적용

plt.title('V14 Distribution by Dataset and Class')
plt.xlabel('Dataset')
plt.ylabel('V14 Value')
plt.legend(title='Class')
plt.tight_layout()
plt.show()

## Save result in submission file 

In [ ]:
submission = pd.read_csv("/kaggle/input/modu-ds-4-credit-card-fraud-detection/sample_submission.csv")

In [ ]:
del submission["Class"]

In [ ]:
submission = submission.merge(test[["id", "Class"]], on="id")

In [ ]:
submission.to_csv("./submission.csv", index=False)